In [1]:
import os

# Check if we're in the root directory of rl-llm repo
if not os.path.basename(os.getcwd()) == 'rl-llm':
    # If we're in a subdirectory of rl-llm, find the root and cd to it
    current_path = os.getcwd()
    while os.path.basename(current_path) != 'rl-llm' and os.path.dirname(current_path) != current_path:
        current_path = os.path.dirname(current_path)
    
    if os.path.basename(current_path) == 'rl-llm':
        os.chdir(current_path)
        print(f"Changed directory to: {current_path}")
    else:
        print("Not in rl-llm repository structure")
else:
    print("Already in rl-llm root directory")

print(f"Current directory: {os.getcwd()}")

Changed directory to: /home/ubuntu/rl-llm
Current directory: /home/ubuntu/rl-llm


In [2]:
from src import EnvManager, sample_episodes
import time
import matplotlib.pyplot as plt
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig

/home/ubuntu/rl-llm/.venv/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment BabyAI-GoTo-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/ubuntu/rl-llm/.venv/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment BabyAI-Open-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/ubuntu/rl-llm/.venv/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment BabyAI-Pickup-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/ubuntu/rl-llm/.venv/lib/python3.10/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment BabyAI-PutNext-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
2025-04-09 18:51:03.465710: E external/

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_id = "pavanpreet-gandhi/babyai-classical-ppo-prefinal-experiments-mix_hard_reason_early_stopping_1_1"
peft_config = PeftConfig.from_pretrained(model_id)
base_model = AutoModelForCausalLM.from_pretrained(peft_config.base_model_name_or_path,device_map="auto")
peft_model = PeftModel.from_pretrained(base_model, model_id)
model = peft_model

tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side='left')
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

generation_kwargs = {
    "max_new_tokens": 20,
    "do_sample": True,
    "top_k": 50,
    "top_p": 0.95,
    "temperature": 0.8,
    "pad_token_id": tokenizer.pad_token_id,
}

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
for env_id in ["BabyAI-GoTo-v0", "BabyAI-Pickup-v0", "BabyAI-PutNext-v0", "BabyAI-Open-v0", "BabyAI-PickUpSeqGoTo"]:

    for reasoning_flag in [False, True]:

        env_ids = [env_id]
        num_envs = 6
        env_managers = [
            EnvManager(
                env_ids, 
                invalid_action_penalty=-2,
                consecutive_invalid_actions_allowed=5,
            )
            for i in range(num_envs)
        ]
        reasoning_flag = reasoning_flag

        start_time = time.time()
        stats = sample_episodes(
            envs=env_managers,
            tokenizer=tokenizer,
            model=model,
            generation_kwargs=generation_kwargs,
            device=device,
            number_of_episodes=20,
            context_window=5,
            reasoning_flag=reasoning_flag,
        )
        elapsed_time = time.time() - start_time
        success_rate = sum(stats["success"]) / len(stats["success"])
        avg_reward = sum(stats["rewards"]) / len(stats["rewards"])
        std_reward = torch.std(torch.tensor(stats["rewards"], dtype=torch.float32)).item()
        avg_episode_length = sum(stats["episode_lengths"]) / len(stats["episode_lengths"])
        std_episode_length = torch.std(torch.tensor(stats["episode_lengths"], dtype=torch.float32)).item()
        avg_num_invalid_actions = sum(stats["num_invalid_actions"]) / len(stats["num_invalid_actions"])

        print(f"Summary: {env_id} with {'Reasoning' if reasoning_flag else 'No Reasoning'}")
        print(f"Sample size: {len(stats['success'])}")
        print(f"Success Rate: {success_rate:.2f}")
        print(f"Average Reward: {avg_reward:.2f} ± {std_reward:.2f}")
        print(f"Average Episode Length: {avg_episode_length:.2f} ± {std_episode_length:.2f}")
        print(f"Average Number of Invalid Actions: {avg_num_invalid_actions:.2f}")
        print()

Summary: BabyAI-GoTo-v0 with No Reasoning
Sample size: 20
Success Rate: 1.00
Average Reward: 0.91 ± 0.05
Average Episode Length: 6.70 ± 3.81
Average Number of Invalid Actions: 0.10

Summary: BabyAI-GoTo-v0 with Reasoning
Sample size: 23
Success Rate: 0.13
Average Reward: 0.12 ± 0.33
Average Episode Length: 5.09 ± 0.79
Average Number of Invalid Actions: 4.70

Summary: BabyAI-Pickup-v0 with No Reasoning
Sample size: 20
Success Rate: 1.00
Average Reward: 0.88 ± 0.05
Average Episode Length: 8.60 ± 4.13
Average Number of Invalid Actions: 0.25

Summary: BabyAI-Pickup-v0 with Reasoning
Sample size: 20
Success Rate: 0.05
Average Reward: 0.04 ± 0.20
Average Episode Length: 5.70 ± 1.69
Average Number of Invalid Actions: 4.90



KeyboardInterrupt: 

## Base model (no finetuning)

Summary: BabyAI-GoTo-v0 with No Reasoning
Sample size: 20
Success Rate: 0.65
Average Reward: 0.51 ± 0.40
Average Episode Length: 23.10 ± 17.94
Average Number of Invalid Actions: 2.15

Summary: BabyAI-GoTo-v0 with Reasoning
Sample size: 20
Success Rate: 0.95
Average Reward: 0.77 ± 0.22
Average Episode Length: 14.80 ± 8.84
Average Number of Invalid Actions: 1.60

Summary: BabyAI-Pickup-v0 with No Reasoning
Sample size: 20
Success Rate: 0.55
Average Reward: 0.43 ± 0.41
Average Episode Length: 28.65 ± 21.20
Average Number of Invalid Actions: 2.30

Summary: BabyAI-Pickup-v0 with Reasoning
Sample size: 20
Success Rate: 0.70
Average Reward: 0.47 ± 0.36
Average Episode Length: 21.30 ± 15.40
Average Number of Invalid Actions: 2.50

Summary: BabyAI-PutNext-v0 with No Reasoning
Sample size: 20
Success Rate: 0.05
Average Reward: 0.01 ± 0.04
Average Episode Length: 78.90 ± 48.60
Average Number of Invalid Actions: 3.65

Summary: BabyAI-PutNext-v0 with Reasoning
Sample size: 21
Success Rate: 0.19
Average Reward: 0.07 ± 0.17
Average Episode Length: 95.81 ± 50.48
Average Number of Invalid Actions: 2.57

Summary: BabyAI-Open-v0 with No Reasoning
Sample size: 20
Success Rate: 0.35
Average Reward: 0.16 ± 0.25
Average Episode Length: 79.35 ± 34.19
Average Number of Invalid Actions: 3.50

Summary: BabyAI-Open-v0 with Reasoning
Sample size: 20
Success Rate: 0.30
Average Reward: 0.15 ± 0.25
Average Episode Length: 81.65 ± 48.86
Average Number of Invalid Actions: 3.05

Summary: BabyAI-PickUpSeqGoTo with No Reasoning
Sample size: 20
Success Rate: 0.55
Average Reward: 0.32 ± 0.32
Average Episode Length: 83.40 ± 39.94
Average Number of Invalid Actions: 0.90

Summary: BabyAI-PickUpSeqGoTo with Reasoning
Sample size: 20
Success Rate: 0.35
Average Reward: 0.19 ± 0.32
Average Episode Length: 84.50 ± 49.86
Average Number of Invalid Actions: 2.90


## Finetuned model (allowed to reason)

